In [104]:
# without dask
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from shapely.ops import unary_union
import numpy as np
import json
import dask.array as da
import dask

# dask arrays are the one which delays the processing here.

with open('overlapping_large_image.json', 'r') as json_file:
    all_data = json.load(json_file)
data = all_data
len(data)


# load all points to one stacknp.max([sublist[1] for sublist in sample['points']])
pop_list = []
merged_polygons = []
sample_element = data[0]
for i, sample in enumerate(data):
    curr_x = np.max([sublist[1] for sublist in sample['points']])
    curr_y = np.max([sublist[1] for sublist in sample['points']])
    for j in range(i+1, len(data)-1):
        given_x = np.max([sublist[0] for sublist in data[j]['points']])
        given_y = np.max([sublist[1] for sublist in data[j]['points']])
        if given_x < curr_x+500 and given_y < curr_y + 500:
            if Polygon(sample['points']).intersects(Polygon(data[j]['points'])):
                merged_polygon = unary_union(
                    [Polygon(sample['points']),
                     Polygon(data[j]['points'])])
                merged_polygons.append(merged_polygon)
                if i not in pop_list:
                    pop_list.append(i)
                if j not in pop_list:
                    pop_list.append(j)


def remove_indexes(lst, indexes):
    # Sort the indexes in descending order
    indexes.sort(reverse=True)

    # Remove the elements at the specified indexes
    for index in indexes:
        if 0 <= index < len(lst):
            del lst[index]

    return lst


new_list = remove_indexes(data, pop_list)

# convert all the data in merged_polygons to json
if merged_polygons:
    for elements in merged_polygons:
        rlist = list(elements.exterior.coords)
        sample_element['points'] = rlist
        data.append(sample_element)

print('number of merged elements ', len(merged_polygons))

#time 4.8s

number of merged elements  0


#### Using multiprocessing without any class structure (14 s)

In [20]:
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from shapely.ops import unary_union
import numpy as np
import json
import multiprocess
from functools import partial  # Import the 'partial' function from functools


def find_overlapping_polygons(sample_idx, data):
    merged_polygons = []
    sample = data[sample_idx]
    curr_x = np.max([sublist[0] for sublist in sample['points']])
    curr_y = np.max([sublist[1] for sublist in sample['points']])
    for j in range(sample_idx + 1, len(data)-1):
        given_x = np.max([sublist[0] for sublist in data[j]['points']])
        given_y = np.max([sublist[1] for sublist in data[j]['points']])
        if given_x < curr_x + 500 and given_y < curr_y + 500:
            if Polygon(sample['points']).intersects(Polygon(data[j]['points'])):
                merged_polygon = unary_union(
                    [Polygon(sample['points']),
                     Polygon(data[j]['points'])])
                merged_polygons.append(merged_polygon)
    return merged_polygons


with open('overlapping_large_image.json', 'r') as json_file:
    data = json.load(json_file)

    # Create a pool of workers
pool = multiprocess.Pool(processes=multiprocess.cpu_count())

# Create a partial function for find_overlapping_polygons with fixed 'data' argument
find_overlapping_partial = partial(find_overlapping_polygons, data=data)

# Use multiprocessing to parallelize the loop for finding overlapping polygons
merged_polygons = pool.map(find_overlapping_partial, range(len(data)))

# Close the pool of workers
pool.close()
pool.join()

# Flatten the list of merged polygons
merged_polygons = [poly for sublist in merged_polygons for poly in sublist]

print('number of merged elements ', len(merged_polygons))

#time 14s

number of merged elements  0


## using bbox method with concurrent futures

In [107]:
# with multiprocessing
from concurrent.futures import ProcessPoolExecutor
from shapely.geometry import Polygon
import numpy as np
import json
from functools import partial

# dask arrays are the one which delays the processing here.

with open('overlapping_large_image.json', 'r') as json_file:
    all_data = json.load(json_file)
data = all_data
print(len(data))


def find_bbox(data, index):
    curr_x_max = np.max([sublist[0] for sublist in data[index]['points']])
    curr_y_max = np.max([sublist[1] for sublist in data[index]['points']])
    curr_x_min = np.min([sublist[0] for sublist in data[index]['points']])
    curr_y_min = np.min([sublist[1] for sublist in data[index]['points']])
    return curr_x_min, curr_x_max, curr_y_min, curr_y_max


def find_overlapping_nuclei_indexes(index):
    curr_x_min = bbbox_list[index][0]
    curr_x_max = bbbox_list[index][1]
    curr_y_min = bbbox_list[index][2]
    curr_y_max = bbbox_list[index][3]
    for j in range(len(bbbox_list)):
        if j == index:
            continue
        given_x_min = bbbox_list[j][0]
        given_x_max = bbbox_list[j][1]
        given_y_min = bbbox_list[j][2]
        given_y_max = bbbox_list[j][3]
        if curr_x_min <= given_x_max and curr_x_max >= given_x_min and curr_y_min <= given_y_max and curr_y_max >= given_y_min:
            return j


def remove_overlapping_nuclei(index):
    for j in range(len(overlapping_bbox)):
        if j == index or j == None:
            continue
        if Polygon(data[index]['points']).intersects(Polygon(data[j]['points'])):
            return j


with ProcessPoolExecutor() as executer:
    bbbox_list = list(executer.map(pfind_bbox, (i for i in range(len(data)))))
    overlapping_bbox = list(
        set(executer.map(find_overlapping_nuclei_indexes, (i for i in range(len(bbbox_list))))))
    nuclei_remove_indexes = list(
        set(executer.map(remove_overlapping_nuclei, (i for i in overlapping_bbox if i is not None))))
    nuclei_remove_indexes_noNone = [i for i in nuclei_remove_indexes if i is not None]


def remove_indexes(lst, indexes):
    # Sort the indexes in descending order
    indexes.sort(reverse=True)

    # Remove the elements at the specified indexes
    for index in indexes:
        if 0 <= index < len(lst):
            del lst[index]

    return lst


new_list = remove_indexes(data, nuclei_remove_indexes_noNone)


print('number of merged elements ', len(nuclei_remove_indexes_noNone))
print('total number of nuclei ', len(all_data))

780


PicklingError: Can't pickle <function find_bbox at 0x7f09515796c0>: it's not the same object as __main__.find_bbox

### Concurrent futures without any class structure (0.4 s)

In [19]:
from concurrent.futures import ProcessPoolExecutor
from shapely.geometry import Polygon
import numpy as np

with open('overlapping_large_image.json', 'r') as json_file:
    all_data = json.load(json_file)


def find_bbox(index):
    curr_x_max = np.max([sublist[0] for sublist in nuclei_list[index]['points']])
    curr_y_max = np.max([sublist[1] for sublist in nuclei_list[index]['points']])
    curr_x_min = np.min([sublist[0] for sublist in nuclei_list[index]['points']])
    curr_y_min = np.min([sublist[1] for sublist in nuclei_list[index]['points']])
    return curr_x_min, curr_x_max, curr_y_min, curr_y_max


def find_overlapping_nuclei_indexes(index):
    curr_x_min = bbbox_list[index][0]
    curr_x_max = bbbox_list[index][1]
    curr_y_min = bbbox_list[index][2]
    curr_y_max = bbbox_list[index][3]
    for j in range(len(bbbox_list)):
        if j == index:
            continue
        given_x_min = bbbox_list[j][0]
        given_x_max = bbbox_list[j][1]
        given_y_min = bbbox_list[j][2]
        given_y_max = bbbox_list[j][3]
        if curr_x_min <= given_x_max and curr_x_max >= given_x_min and curr_y_min <= given_y_max and curr_y_max >= given_y_min:
            return j


def remove_overlapping_nuclei(index):
    for j in range(len(overlapping_bbox)):
        if j == index or j == None:
            continue
        if Polygon(nuclei_list[index]['points']).intersects(Polygon(nuclei_list[j]['points'])):
            return j


def remove_indexes(lst, indexes):
    # Sort the indexes in descending order
    indexes.sort(reverse=True)

    # Remove the elements at the specified indexes
    for index in indexes:
        if 0 <= index < len(lst):
            del lst[index]

    return lst


def remove_overlap_nuclei(nuclei_list):

    print('>> starting overlap nuclei removal ')

    with ProcessPoolExecutor() as executer:
        bbbox_list = list(executer.map(find_bbox, (i for i in range(len(nuclei_list)))))
        overlapping_bbox = list(
            set(executer.map(find_overlapping_nuclei_indexes, (i for i in range(len(bbbox_list))))))
        nuclei_remove_indexes = list(
            set(executer.map(remove_overlapping_nuclei, (i for i in overlapping_bbox if i is not None))))
        nuclei_remove_indexes_noNone = [i for i in nuclei_remove_indexes if i is not None]

    new_list = remove_indexes(nuclei_list, nuclei_remove_indexes_noNone)
    print('>> Number of nuclei after overlap removal ', len(new_list))

    return new_list

nuclei_list = remove_overlap_nuclei(nuclei_list)


>> starting overlap nuclei removal 


NameError: name 'bbbox_list' is not defined

### Using pathos for multiprocessing with Bbox method (5 s)

In [15]:
# using pathos

from pathos.multiprocessing import ProcessingPool
from shapely.geometry import Polygon
import numpy as np


def remove_overlap_nuclei(nuclei_list):

    print('>> starting overlap nuclei removal ')

    def _find_bbox(index):
        curr_x_max = np.max([sublist[0] for sublist in nuclei_list[index]['points']])
        curr_y_max = np.max([sublist[1] for sublist in nuclei_list[index]['points']])
        curr_x_min = np.min([sublist[0] for sublist in nuclei_list[index]['points']])
        curr_y_min = np.min([sublist[1] for sublist in nuclei_list[index]['points']])
        return curr_x_min, curr_x_max, curr_y_min, curr_y_max

    def _remove_overlapping_nuclei(index):
        for j in range(len(overlapping_bbox)):
            if j == index or j == None:
                continue
            if Polygon(nuclei_list[index]['points']).intersects(Polygon(nuclei_list[j]['points'])):
                return j

    def _remove_indexes(lst, indexes):
        # Sort the indexes in descending order
        indexes.sort(reverse=True)

        # Remove the elements at the specified indexes
        for index in indexes:
            if 0 <= index < len(lst):
                del lst[index]

        return lst

    def _find_overlapping_nuclei_indexes(index):
        curr_x_min = bbbox_list[index][0]
        curr_x_max = bbbox_list[index][1]
        curr_y_min = bbbox_list[index][2]
        curr_y_max = bbbox_list[index][3]
        for j in range(len(bbbox_list)):
            if j == index:
                continue
            given_x_min = bbbox_list[j][0]
            given_x_max = bbbox_list[j][1]
            given_y_min = bbbox_list[j][2]
            given_y_max = bbbox_list[j][3]
            if curr_x_min <= given_x_max and curr_x_max >= given_x_min and curr_y_min <= given_y_max and curr_y_max >= given_y_min:
                return j

    pool = ProcessingPool(2)
    bbbox_list = list(pool.map(_find_bbox, (i for i in range(len(nuclei_list)))))
    overlapping_bbox = list(
        set(pool.map(_find_overlapping_nuclei_indexes, (i for i in range(len(bbbox_list))))))
    nuclei_remove_indexes = list(
        set(pool.map(_remove_overlapping_nuclei, (i for i in overlapping_bbox if i is not None))))
    nuclei_remove_indexes_noNone = [i for i in nuclei_remove_indexes if i is not None]

    new_list = _remove_indexes(nuclei_list, nuclei_remove_indexes_noNone)
    print('>> Number of nuclei after overlap removal ', len(new_list))

    return new_list

with open('overlapping_large_image.json', 'r') as json_file:
    nuclei_list = json.load(json_file)
nuclei_list = remove_overlap_nuclei(nuclei_list)

#time 4.8 s

>> starting overlap nuclei removal 


>> Number of nuclei after overlap removal  780


### using concurrent.futures without class structure with strTree (0.4 s)

In [22]:
from concurrent.futures import ProcessPoolExecutor
from shapely.geometry import Polygon
import shapely
import numpy as np

with open('overlapping_large_image.json', 'r') as json_file:
    nuclei_list = json.load(json_file)

def create_polygons(index):
    return Polygon(nuclei_list[index]['points'])

def remove_indexes(lst, indexes):
        # Sort the indexes in descending order
        indexes.sort(reverse=True)

        # Remove the elements at the specified indexes
        for index in indexes:
            if 0 <= index < len(lst):
                del lst[index]

        return lst

with ProcessPoolExecutor() as executor:
    polys = list((executor.map(create_polygons, (i for i in range(len(nuclei_list))))))
rt = shapely.strtree.STRtree(polys)
remove_list = [index for index in range(len(polys)) if any(ix for ix in rt.query(polys[index]) if ix < index and polys[index].intersects(polys[ix]))]
output_list = remove_indexes(nuclei_list,remove_list)
print(len(nuclei_list))
print(remove_list)

#time 0.3s

780
[]


### Using conucrrent futures with strTree, Class structure (17 s)

In [12]:
from concurrent.futures import ProcessPoolExecutor
from shapely.geometry import Polygon
import shapely.strtree

class NucleiProcessor:

    def __init__(self, nuclei_list):
        self.nuclei_list = nuclei_list

    def create_polygons(self, index):
        return Polygon(self.nuclei_list[index]['points'])

    def remove_indexes(self, lst, indexes):
        # Sort the indexes in descending order
        indexes.sort(reverse=True)

        # Remove the elements at the specified indexes
        for index in indexes:
            if 0 <= index < len(lst):
                del lst[index]

        return lst

    def process_nuclei(self):
        with ProcessPoolExecutor() as executor:
            polys = list(executor.map(self.create_polygons, (i for i in range(len(self.nuclei_list)))))

        rt = shapely.strtree.STRtree(polys)
        remove_list = [index for index in range(len(polys)) if any(ix for ix in rt.query(polys[index]) if ix < index and polys[index].intersects(polys[ix]))]

        output_list = self.remove_indexes(self.nuclei_list, remove_list)

        return output_list



with open('overlapping_large_image.json', 'r') as json_file:
    nuclei_list = json.load(json_file)
processor = NucleiProcessor(nuclei_list)
output_list = processor.process_nuclei()
print(len(output_list),len(nuclei_list))

#time 17s

780 780


### using pathos multiprocessing with strTree (0.2 second)

In [7]:
from pathos.multiprocessing import ProcessingPool
from shapely.geometry import Polygon

with open('overlapping_large_image.json', 'r') as json_file:
    nuclei_list = json.load(json_file)

def create_polygons(index):
    return Polygon(nuclei_list[index]['points'])

def remove_indexes(lst, indexes):
        # Sort the indexes in descending order
        indexes.sort(reverse=True)

        # Remove the elements at the specified indexes
        for index in indexes:
            if 0 <= index < len(lst):
                del lst[index]

        return lst

pool = ProcessingPool()
polys = list((pool.map(create_polygons, (i for i in range(len(nuclei_list))))))
print(len(polys))
rt = shapely.strtree.STRtree(polys)
remove_list = [index for index in range(len(polys)) if any(ix for ix in rt.query(polys[index]) if ix < index and polys[index].intersects(polys[ix]))]
output_list = remove_indexes(nuclei_list,remove_list)
print(remove_list)
#time 0.2s

780
[]
